[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
### GETS THE DATA FOR THE TIMECUMCASES GRAPH
filters = ['areaType=nation', 'areaName=England']
structure = {
    'date' : 'date',
    'newCases' : 'newCasesByPublishDate',
    'cumCases' : 'cumCasesByPublishDate',
    'newDeaths' : 'newDeaths28DaysByDeathDate',
    'cumDeaths' : 'cumDeaths28DaysByDeathDate',
    'males' : 'maleCases',
    'females' : 'femaleCases'
}
api = Cov19API(filters=filters, structure=structure)
timecumcases = api.get_json()
mf_cases = api.get_json()

In [4]:
#print(timecumcases)

In [5]:
#print(mf_cases)

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [6]:
### OPENS TIMECUMCASES JSON
with open ('timecumcases.json', 'rt') as INFILEtcc:
    data = json.load(INFILEtcc)

In [7]:
### CREATING THE DATALIST
datalist = data['data']
#print(data)

In [8]:
### SORTING BY DATE
dates = [dictionary['date'] for dictionary in datalist]
dates.sort()

In [9]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format= '%Y-%m-%d')

In [10]:
### SHOWS THE FORMAT OF THE DATE LIST
startdate = parse_date(dates[0])
enddate = parse_date(dates[-1])
print(startdate, ' to ', enddate)

2020-01-03 00:00:00  to  2020-11-25 00:00:00


In [11]:
### OPENS MF_CASES JSON
with open ('mf_cases.json', 'rt') as INFILEmf:
    data = json.load(INFILEmf)

In [12]:

datadic = data['data'][1]
#print(datadic)

males = datadic['males']
females = datadic['females']
ageranges = []


ageranges = [x['age'] for x in males]
#print(ageranges)

In [13]:
def min_age(agerange):
    agerange = agerange.replace('+', '')
    start = agerange.split('_')[0]
    return int(start)

ageranges.sort(key = min_age)
#print(ageranges)

In [14]:
mf_casesdf = pd.DataFrame(index = ageranges, columns = ['males', 'females', 'total'])

In [15]:
### SAVES DATA FOR BOTH GRAPHS AS JSON FILES
with open('timecumcases.json', 'wt') as OUTFILEtcc:
    json.dump(timecumcases, OUTFILEtcc)
with open('timecumcases.json', 'wt') as OUTFILEmf:
    json.dump(mf_cases, OUTFILEmf)


## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [16]:
index = pd.date_range(startdate, enddate, freq='D')
timecumcasesdf = pd.DataFrame(index = index, columns = ['newCases', 'cumCases', 'newDeaths', 'cumDeaths'])

def wrangle_data(rawdata):
    """Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in."""
    for entry in datalist:
        date = parse_date(entry['date'])
    for column in [
        'newCases', 'cumCases', 'newDeaths','cumDeaths']:
        if pd.isna(timecumcasesdf.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            timecumcasesdf.loc[date, column] = value
    timecumcasesdf.fillna(0.0, inplace = True)
    return timecumcasesdf


timecumcasesdf = wrangle_data(timecumcasesdf)
#timecumcasesdf

In [17]:
def mf_wrangle(rawdata):
    for entry in males:
        ageband = entry['age']
        mf_casesdf.loc[ageband, 'males'] = entry['value']

    for entry in females:
        ageband = entry['age']
        mf_casesdf.loc[ageband, 'females'] = entry['value']
    return mf_casesdf, ageband

mf_casesdf['total'] = mf_casesdf['males'] + mf_casesdf['females']
#print(mf_casesdf)

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [18]:
# Button to access API
def access_api(button):
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    print("I'm downloading data from the API...")
    print("... all done.")


In [19]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
    
apibutton = wdg.Button(description = 'Refresh Data',
                      disabled = False,
                      button_style = 'info', 
                      tooltip = 'Click to download the current Public Health england data',
                      icon = 'download' 
                      )

            # register the callback with the button
apibutton.on_click(access_api)

            # used to display the widgets
#display(apibutton)

In [20]:
### SAVES DATA FOR BOTH GRAPHS AS JSON FILES
with open('timecumcases.json', 'wt') as OUTFILEtcc:
    json.dump(timecumcases, OUTFILEtcc)
with open('mf_cases.json', 'wt') as OUTFILEmf:
    json.dump(mf_cases, OUTFILEmf)


## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [21]:
timecumcasesdf = pd.read_pickle('timecumcasesdf.pkl')

series = wdg.SelectMultiple(
    options = [
        'newCases', 'cumCases', 'newDeaths','cumDeaths'], 
    value = [
        'newCases', 'cumCases', 'newDeaths','cumDeaths'], 
    rows = 3, 
    description = 'Stats: ',
    disabled = False
)

scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    # defaults to linear
    # layout = {'width' : 'max content'}, if the items names are long
    description = 'Scale',
    disabled = False
)

# HBox = Horizzontal box, VBox = vertical box
controls = wdg.VBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale == 'Linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        timecumcasesdf[list(gcols)].plot(logy = logscale)
        plt.title('Cases/Deaths Graph')
        plt.xlabel('Time')
        plt.ylabel('Number of cases')
    else:
        print('Click to start data for graph...')
        print('(CNTRL-Click to select more than one category)')

graph = wdg.interactive_output(timeseries_graph, 
                               {'gcols': series, 'gscale': scale})

ctrls = wdg.VBox([series, scale, apibutton])

form = wdg.HBox([graph, ctrls])

display(form)

In [22]:
mf_casesdf = pd.read_pickle('mfcasesdf.pkl')

agecols = wdg.SelectMultiple(
    options = ['males', 'females', 'total'], 
    value = ['males', 'females'],
    rows = 3,
    description = 'Sex',
    disabled = False
)

def age_graph(graphcolumns):
    # our callback function
    ncols = len(graphcolumns)
    if ncols > 0:
        mf_casesdf.plot(kind = 'bar',
                   y = list(graphcolumns)
                   )
        plt.title('Male/Female Cases')
        plt.xlabel('Age Range')
        plt.ylabel('Number of cases')
    else:
        #if the user has not selected a column, print a message instead
        print('Click to start data for graph...')
        print('(CNTRL-Click to select more than one category)')
        
# keep calling age_graph(graphcolumns = value_of_agecols)
# capture output in variable output
output = wdg.interactive_output(age_graph, {'graphcolumns': agecols})

form2 = wdg.VBox([agecols, apibutton])

display = wdg.HBox([output, form2])
display

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*